In [1]:
from google.colab import files
uploaded = files.upload()

Saving 01 Harry Potter and the Sorcerers Stone.txt to 01 Harry Potter and the Sorcerers Stone.txt


In [4]:
filename = "01 Harry Potter and the Sorcerers Stone.txt"

with open(filename, 'r', encoding='utf-8') as f:
    content = f.read()

print("Total number of characters are" , len(content))

print(content[:999])


Total number of characters are 439478
M r. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were the last people you’d expect to be involved in anything strange or mysterious, because they just didn’t hold with such nonsense.

Mr. Dursley was the director of a firm called Grunnings, which made drills. He was a big, beefy man with hardly any neck, although he did have a very large mustache. Mrs. Dursley was thin and blonde and had nearly twice the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbors. The Dursleys had a small son called Dudley and in their opinion there was no finer boy anywhere.

The Dursleys had everything they wanted, but they also had a secret, and their greatest fear was that somebody would discover it. They didn’t think they could bear it if anyone found out about the Potters. Mrs. Potter was Mrs. Dursley’s sist

Our goal is to tokenize this 2439478-character short story into individual words and special characters that we can then turn into embeddings for LLM training

To do so we are going to use re library

In [6]:
import re
text = "Hello dear, My name is Ashu."
result = re.split(r'([,.]|\s)',text)
print(result)


['Hello', ' ', 'dear', ',', '', ' ', 'My', ' ', 'name', ' ', 'is', ' ', 'Ashu', '.', '']


A small remaining issue is that the list still includes whitespace characters. Optionally, we can remove these redundant characters safely as follows:


In [7]:
result = [item for item in result if item.strip()]
print(result)

['Hello', 'dear', ',', 'My', 'name', 'is', 'Ashu', '.']


REMOVING WHITESPACES OR NOT

When developing a simple tokenizer, whether we should encode whitespaces as separate characters or just remove them depends on our application and its requirements. Removing whitespaces reduces the memory and computing requirements. However, keeping whitespaces can be useful if we train models that are sensitive to the exact structure of the text (for example, Python code, which is sensitive to indentation and spacing). Here, we remove whitespaces for simplicity and brevity of the tokenized outputs. Later, we will switch to a tokenization scheme that includes whitespaces.

The tokenization scheme we devised above works well on the simple sample text. Let's modify it a bit further so that it can also handle other types of punctuation, such as question marks, quotation marks, and the double-dashes

In [11]:
text = "hello world! IS this-- Ashu?"
result = re.split(r'([,.;:_?!"()\']|--|\s)' , text)
result = [item for item in result if item.strip()]
print(result)

['hello', 'world', '!', 'IS', 'this', '--', 'Ashu', '?']


Now that we got a basic tokenizer working, let's apply it to Harry Potter's entire story:

In [14]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', content)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:300])


['M', 'r', '.', 'and', 'Mrs', '.', 'Dursley', ',', 'of', 'number', 'four', ',', 'Privet', 'Drive', ',', 'were', 'proud', 'to', 'say', 'that', 'they', 'were', 'perfectly', 'normal', ',', 'thank', 'you', 'very', 'much', '.', 'They', 'were', 'the', 'last', 'people', 'you’d', 'expect', 'to', 'be', 'involved', 'in', 'anything', 'strange', 'or', 'mysterious', ',', 'because', 'they', 'just', 'didn’t', 'hold', 'with', 'such', 'nonsense', '.', 'Mr', '.', 'Dursley', 'was', 'the', 'director', 'of', 'a', 'firm', 'called', 'Grunnings', ',', 'which', 'made', 'drills', '.', 'He', 'was', 'a', 'big', ',', 'beefy', 'man', 'with', 'hardly', 'any', 'neck', ',', 'although', 'he', 'did', 'have', 'a', 'very', 'large', 'mustache', '.', 'Mrs', '.', 'Dursley', 'was', 'thin', 'and', 'blonde', 'and', 'had', 'nearly', 'twice', 'the', 'usual', 'amount', 'of', 'neck', ',', 'which', 'came', 'in', 'very', 'useful', 'as', 'she', 'spent', 'so', 'much', 'of', 'her', 'time', 'craning', 'over', 'garden', 'fences', ',', 'sp

In [15]:
print(len(preprocessed))

93318


**Step 2: Creating Token IDs**

In [16]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
vocab_size

7572

After determining that the vocabulary size is 7572 via the above code, we create the vocabulary and print its first 100 entries for illustration purposes:

In [17]:
vocab = {token:integer for integer,token in enumerate(all_words)}

In [19]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 99:
        break

('!', 0)
("'", 1)
('(', 2)
(')', 3)
('*', 4)
(',', 5)
('-', 6)
('-bodied', 7)
('.', 8)
('1', 9)
('1473', 10)
('1637', 11)
('17', 12)
('1709', 13)
('1945', 14)
('2', 15)
('3', 16)
('31', 17)
('382', 18)
('4', 19)
(':', 20)
(';', 21)
('?', 22)
('A', 23)
('ALBUS', 24)
('ALLEY', 25)
('ALLOWED', 26)
('AM', 27)
('AND', 28)
('ANYTHING', 29)
('ARE', 30)
('AT', 31)
('About', 32)
('According', 33)
('Adalbert', 34)
('Add', 35)
('Adrian', 36)
('Africa', 37)
('African', 38)
('After', 39)
('Against', 40)
('Ages', 41)
('Agrippa', 42)
('Ah', 43)
('Ahead', 44)
('Alberic', 45)
('Albus', 46)
('Albus…”', 47)
('Algie', 48)
('Alicia', 49)
('All', 50)
('Alley', 51)
('Alohomora', 52)
('Although', 53)
('Always', 54)
('An', 55)
('And', 56)
('Angelina', 57)
('Another', 58)
('Anti-Dark', 59)
('Anticheating', 60)
('Any', 61)
('Anyone', 62)
('Anything', 63)
('Anythin’', 64)
('Anyway', 65)
('An’', 66)
('Apothecary', 67)
('Are', 68)
('Argus', 69)
('Around', 70)
('Arsenius', 71)
('Arts', 72)
('As', 73)
('Astronomy', 7

 We want to convert the outputs of an LLM from numbers back into text, we also need a way to turn token IDs into text.

For this, we can create an inverse version of the vocabulary that maps token IDs back to corresponding text tokens.

To do so we define a class for tokenisers in pyhton

In [20]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)

        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

Let's instantiate a new tokenizer object from the SimpleTokenizerV1 class and tokenize a passage from Edith Wharton's short story to try it out in practice:

In [26]:
tokenizer = SimpleTokenizerV1(vocab)
text = "Mr. Dursley was the director of a firm called Grunnings, which made drills. He was a big, beefy man with hardly any neck,"

ids = tokenizer.encode(text)
print(ids)

[735, 8, 292, 6597, 6131, 2531, 4452, 1300, 2991, 1954, 472, 5, 6683, 4086, 2650, 8, 515, 6597, 1300, 1692, 5, 1633, 4100, 6770, 3468, 1431, 4354, 5]


In [27]:
tokenizer.decode(ids)

'Mr. Dursley was the director of a firm called Grunnings, which made drills. He was a big, beefy man with hardly any neck,'

So far, so good. We implemented a tokenizer capable of tokenizing and de-tokenizing text based on a snippet from the training set.

Let's now apply it to a new text sample that is not contained in the training set:

In [29]:
text = "Harry, do you know about blockchain?"
print(tokenizer.encode(text))

KeyError: 'blockchain'

The problem is that the word "Blockchain" was not used in the Harry Potter story.

Hence, it is not contained in the vocabulary.

This highlights the need to consider large and diverse training sets to extend the vocabulary when working on LLMs.

#ADDING SPECIAL CONTEXT TOKENS
In the previous section, we implemented a simple tokenizer and applied it to a passage from the training set.

In this section, we will modify this tokenizer to handle unknown words.

In particular, we will modify the vocabulary and tokenizer we implemented in the previous section, SimpleTokenizerV2, to support two new tokens, <|unk|> and <|endoftext|>

We can modify the tokenizer to use an <|unk|> token if it encounters a word that is not part of the vocabulary.

Furthermore, we add a token between unrelated texts.

For example, when training GPT-like LLMs on multiple independent documents or books, it is common to insert a token before each document or book that follows a previous text source

Let's now modify the vocabulary to include these two special tokens, and <|endoftext|>, by adding these to the list of all unique words that we created in the previous section:

In [33]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [34]:
print(len(all_tokens))

7574


Based on the output of the print statement above, the new vocabulary size is 7574 (the vocabulary size in the previous section was 7572).

As an additional quick check, let's print the last 5 entries of the updated vocabulary:

In [35]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('…Oak', 7569)
('…Then', 7570)
('…”', 7571)
('<|endoftext|>', 7572)
('<|unk|>', 7573)


A simple text tokenizer that handles unknown words

Step 1: Replace unknown words by <|unk|> tokens

Step 2: Replace spaces before the specified punctuations

In [36]:

class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [37]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Harry, do you know about blockchain?"
text2 = "The Dursleys had everything they wanted, but they also had a secret, and their greatest fear was that somebody would discover it."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Harry, do you know about blockchain? <|endoftext|> The Dursleys had everything they wanted, but they also had a secret, and their greatest fear was that somebody would discover it.


In [38]:
tokenizer.encode(text)

[508,
 5,
 2569,
 6871,
 3867,
 1306,
 7573,
 22,
 7572,
 1113,
 293,
 3418,
 2817,
 6144,
 6580,
 5,
 1934,
 6144,
 1387,
 3418,
 1300,
 5272,
 5,
 1404,
 6132,
 3360,
 2922,
 6597,
 6126,
 5625,
 6817,
 2542,
 3760,
 8]

In [39]:
tokenizer.decode(tokenizer.encode(text))

'Harry, do you know about <|unk|>? <|endoftext|> The Dursleys had everything they wanted, but they also had a secret, and their greatest fear was that somebody would discover it.'

Based on comparing the de-tokenized text above with the original input text, we know that the training dataset, Harry Potter story The Verdict, did not contain the words "blockchain".

So far, we have discussed tokenization as an essential step in processing text as input to LLMs. Depending on the LLM, some researchers also consider additional special tokens such as the following:

[BOS] (beginning of sequence): This token marks the start of a text. It signifies to the LLM where a piece of content begins.

[EOS] (end of sequence): This token is positioned at the end of a text, and is especially useful when concatenating multiple unrelated texts, similar to <|endoftext|>. For instance, when combining two different Wikipedia articles or books, the [EOS] token indicates where one article ends and the next one begins.

[PAD] (padding): When training LLMs with batch sizes larger than one, the batch might contain texts of varying lengths. To ensure all texts have the same length, the shorter texts are extended or "padded" using the [PAD] token, up to the length of the longest text in the batch.

Note that the tokenizer used for GPT models does not need any of these tokens mentioned above but only uses an <|endoftext|> token for simplicity

the tokenizer used for GPT models also doesn't use an <|unk|> token for outof-vocabulary words. Instead, GPT models use a byte pair encoding tokenizer, which breaks down words into subword units